In [1]:
import mlflow

import mlflow.sklearn

import pandas as pd

import matplotlib.pyplot as plt

import keras

import mlflow.keras

from sklearn.model_selection import train_test_split

from sklearn.datasets import fetch_california_housing

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

2023-01-10 02:02:09.515594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 02:02:09.644739: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-10 02:02:09.644752: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-10 02:02:14.444386: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
model = ResNet50(weights='imagenet')

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

print('Predicted:', decode_predictions(preds, top=3)[0])

with mlflow.start_run():
    active_run = mlflow.active_run()
    mlflow.keras.save_model(
        model,
        "Org-model-1"
        )

    print("Active run_id: %s",active_run.info.run_id)
    
    artifacts = {
    "Original_model": "Org-model-1"
    }

    model_data = mlflow.pyfunc.log_model(
        artifact_path="ML-model-1",
        python_model=wrapper.TestWrapper(),
        artifacts=artifacts,
        code_path = ["wrapper.py"],
        )

    registered_model_name="digit-classification"
    mv = mlflow.register_model(model_data.model_uri, registered_model_name)

NameError: name 'ResNet50' is not defined

In [4]:
import numpy as np
import mlflow
import torch
from torch import nn
import torch
import urllib
from PIL import Image
from torchvision import transforms


url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)


In [7]:
b_model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

with mlflow.start_run() as run:
    mlflow.pytorch.save_model(b_model, "model")

model = mlflow.pytorch.load_model("model")
model.eval()

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Using cache found in /home/cliquot/.cache/torch/hub/pytorch_vision_v0.10.0


Samoyed 0.9378382563591003
Pomeranian 0.008283428847789764
Great Pyrenees 0.005603063385933638
Arctic fox 0.005527763161808252
white wolf 0.004741044249385595
